In [2]:
pip install transformers

     ---------------------------------------- 9.9/9.9 MB 27.5 MB/s eta 0:00:00
     ------------------------------------- 286.1/286.1 kB 17.2 MB/s eta 0:00:00
     ---------------------------------------- 436.6/436.6 kB ? eta 0:00:00
     ---------------------------------------- 2.4/2.4 MB 37.7 MB/s eta 0:00:00
     ------------------------------------- 179.3/179.3 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1
Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np

from transformers import BertTokenizer,BertForSequenceClassification

model_path = "./saved_model"
loaded_model = BertForSequenceClassification.from_pretrained(model_path)
loaded_tokenizer = BertTokenizer.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)

def analyze_sentiment(news_text, model, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(news_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=-1).item()
    sentiment = ['negative', 'neutral', 'positive'][pred]
    confidence = probs[0][pred].item()
    return sentiment, confidence

news = "삼성전자의 실적이 예상을 뛰어넘어 주가가 상승했습니다."
sentiment, confidence = analyze_sentiment(news, loaded_model, loaded_tokenizer)
print(f"뉴스: {news}")
print(f"감성: {sentiment}")
print(f"신뢰도: {confidence:.2f}")

뉴스: 삼성전자의 실적이 예상을 뛰어넘어 주가가 상승했습니다.
감성: positive
신뢰도: 1.00


In [4]:
news = "여기에 뉴스기사 제목을 넣어주시면 됩니다."
sentiment, confidence = analyze_sentiment(news, loaded_model, loaded_tokenizer)
print(f"뉴스: {news}")
print(f"감성: {sentiment}")
print(f"신뢰도: {confidence:.2f}")

뉴스: 여기에 뉴스기사 제목을 넣어주시면 됩니다.
감성: neutral
신뢰도: 1.00


In [27]:
import os
import sys
import urllib.request
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
from datetime import datetime
import time
start_time = time.time()  # 코드 실행 전 시간 측정

client_id = "VchsEEGhs2hkE9ozLcmM"
client_secret = "_xJqlJO5uJ"

def get_top_kospi_companies():
    company_list_df = pd.DataFrame(columns=('id','name'))
    company_id = 1
    # 2페이지까지 시총 상위 100대 기업이 표시됨
    for page in range(1,3):
        url = f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={page}"
        response = requests.get(url)
        if response.status_code != 200:
            print("Failed to request:", response.status_code)
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find("table", class_="type_2")
        tbody = table.find("tbody")
        rows = tbody.find_all("tr", onmouseover="mouseOver(this)")

        for row in rows:
            cols = row.find_all("td")
            if cols and len(company_list_df) < 100:
                company_name = cols[1].text.strip()
                company_list_df.loc[company_id] = [company_id, company_name]
                company_id += 1
            else:
                return company_list_df

    return company_list_df

def get_news(row):
    global news_df, news_index

    id = row['id']
    company_name = row['name']
    print(news_index, company_name)
    company = company_name
    query = urllib.parse.quote(company)
    display = 100

    url = "https://openapi.naver.com/v1/search/news?query=" + query +"&display=" + str(display) + "&sort=date" # JSON 결과

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read().decode('utf-8')
        response_dict = json.loads(response_body)
        items = response_dict['items']
        for item_index in range(0, len(items)):
            remove_tag = re.compile(r'<.*?>|\[.*?\]')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            title = title.replace('&quot;', '"')
            if title=='':
                continue
            link = items[item_index]['link']
            date = datetime.strptime(items[item_index]['pubDate'], "%a, %d %b %Y %H:%M:%S %z")
            date = date.strftime("%Y-%m-%d %H:%M")
            sentiment, confidence = analyze_sentiment(title, loaded_model, loaded_tokenizer)
            news_df.loc[news_index] = [id, title, link, date, sentiment, confidence]
            news_index += 1  # 인덱스 업데이트
    else:
        print("Error Code:" + rescode)

# 코스피 시총 상위 100대 기업 불러오기
company_list_df = get_top_kospi_companies()
company_list_df.to_csv('company_list.csv', encoding='utf-8-sig', index=False)

# 각 기업별 네이버뉴스 최근 기사 제목 불러오기 (최대 100개)
news_df = pd.DataFrame(columns=('comp_id', 'title', 'link', 'date', 'sentiment','confidence'))
news_index = 0  # 뉴스 인덱스를 0으로 초기화
company_list_df.apply(get_news, axis=1)
news_df.to_csv('news.csv', encoding='utf-8-sig', index=False)

# 실행시간 측정
end_time = time.time()
print("Execution time: {:.6f} seconds".format(end_time - start_time))

0 삼성전자
100 SK하이닉스
200 LG에너지솔루션
300 삼성바이오로직스
400 현대차
500 셀트리온
600 삼성전자우
700 기아
800 KB금융
900 신한지주
1000 POSCO홀딩스
1100 NAVER
1200 LG화학
1300 삼성물산
1400 삼성SDI
1500 현대모비스
1600 삼성생명
1700 메리츠금융지주
1800 하나금융지주
1900 포스코퓨처엠
2000 한화에어로스페이스
2100 HD현대중공업
2200 고려아연
2300 삼성화재
2400 카카오
2500 크래프톤
2600 LG전자
2700 KT&G
2800 HD한국조선해양
2900 한국전력
3000 HMM
3100 유한양행
3200 두산에너빌리티
3300 LG
3399 우리금융지주
3499 SK텔레콤
3599 기업은행
3699 삼성에스디에스
3798 SK스퀘어
3898 HD현대일렉트릭
3998 SK
4098 카카오뱅크
4198 KT
4298 SK이노베이션
4398 한미반도체
4498 SK바이오팜
4598 한화오션
4698 포스코인터내셔널
4798 현대글로비스
4898 삼성전기
4998 삼성중공업
5098 KODEX CD금리액티브(합성)
5198 DB손해보험
5298 대한항공
5398 에코프로머티
5498 하이브
5598 아모레퍼시픽
5698 현대로템
5798 S-Oil
5898 TIGER CD금리투자KIS(합성)
5998 HD현대
6098 현대차2우B
6198 한진칼
6298 KODEX 200
6398 SKC
6498 LG생활건강
6598 LIG넥스원
6698 미래에셋증권
6798 LG디스플레이
6898 맥쿼리인프라
6998 넷마블
7098 HD현대마린솔루션
7198 한국항공우주
7298 코웨이
7398 TIGER 미국S&P500
7498 LG이노텍
7598 삼성카드
7698 한국타이어앤테크놀로지
7798 LS ELECTRIC
7898 NH투자증권
7998 KODEX KOFR금리액티브(합성)
8098 CJ제일제당
8198 SK바이오사이언스
8298 한국금융지주
8398 LG유플러스


In [28]:
news = ' 미래에셋, 2차전지·전기차 ETF 7종 순자산 4조원 돌파'
sentiment, confidence = analyze_sentiment(news, loaded_model, loaded_tokenizer)
print(f"뉴스: {news}")
print(f"감성: {sentiment}")
print(f"신뢰도: {confidence:.2f}")

뉴스:  미래에셋, 2차전지·전기차 ETF 7종 순자산 4조원 돌파
감성: positive
신뢰도: 0.60


In [8]:
import pandas as pd

# 코스피 시총 상위 100대 기업 불러오기
company_list_df = pd.read_csv('company_list.csv', encoding='utf-8-sig')

company_list_df.head()

,id,name
0,1,삼성전자
1,2,SK하이닉스
2,3,LG에너지솔루션
3,4,삼성바이오로직스
4,5,현대차


In [12]:
# 각 기업별 네이버뉴스 최근 기사 제목 불러오기 (최대 100개)
news_data = pd.read_csv('news.csv', encoding='utf-8-sig')
news_df.columns = ['comp_id', 'title', 'link', 'date', 'sentiment', 'confidence']

news_df.head()

,comp_id,title,link,date,sentiment,confidence
0,1,"'사면초가 위기' 삼성전자, 지배구조 리스크 해소할까",https://www.straightnews.co.kr/news/articleVie...,2024-10-15 18:06,positive,0.942887
1,1,기관이 돌아왔다… 코스피지수 상승 상품에 '베팅',https://n.news.naver.com/mnews/article/014/000...,2024-10-15 18:06,neutral,0.800807
2,1,삼성과 LG는 '가전' SKT는 'UAM'.. AI를 통한 스마트한 변화에...,https://www.techm.kr/news/articleView.html?idx...,2024-10-15 18:04,neutral,0.991741
3,1,"아나배틱세미 정세웅 CEO, 시리즈A 투자 유치 성공",https://www.esgeconomy.com/news/articleView.ht...,2024-10-15 18:04,positive,0.990037
4,1,"코스피, 외인·기관 동반 매수세에 2630선 '회복'",http://www.betanews.net:8080/article/1477034.html,2024-10-15 18:02,positive,0.986578


In [39]:
import pandas as pd
from sqlalchemy import create_engine, text, Column, Integer, String, DateTime, Float, Enum, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.exc import SQLAlchemyError

Base = declarative_base()

class CompanyList(Base):
    __tablename__ = 'company_list'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    news = relationship("NewsData", back_populates="company")

class NewsData(Base):
    __tablename__ = 'news_data'
    id = Column(Integer, primary_key=True, autoincrement=True)
    comp_id = Column(Integer, ForeignKey('company_list.id'))  # company_list 테이블과의 외래키 관계
    title = Column(String)
    link = Column(String)
    date = Column(DateTime)
    sentiment = Column(Enum('positive', 'negative', 'neutral'))
    confidence = Column(Float)
    company = relationship("CompanyList", back_populates="news")

# 데이터베이스 연결 설정
database_username = 'user'
database_password = '12345678'
database_ip       = '34.67.218.11'
database_name     = 'sentimentdb'
database_connection = f'mysql+mysqlconnector://{database_username}:{database_password}@{database_ip}/{database_name}'

# 데이터베이스 엔진 생성
engine = create_engine(database_connection)
Session = sessionmaker(bind=engine)

# 트랜잭션 시작 및 처리
session = Session()
try:
    # 외래 키 검사 비활성화
    session.execute(text("SET FOREIGN_KEY_CHECKS=0;"))

    # 기존 데이터 삭제
    session.execute(text("DELETE FROM company_list;"))
    session.execute(text("DELETE FROM news_data;"))
    session.execute(text("ALTER TABLE news_data AUTO_INCREMENT = 1;"))  # AUTO_INCREMENT 재설정

    # 데이터 벌크 삽입
    print(1)
    session.bulk_insert_mappings(CompanyList, company_list_df.to_dict(orient='records'))
    print(2)
    session.bulk_insert_mappings(NewsData, news_df.to_dict(orient='records'))

    # 외래 키 검사 활성화
    session.execute(text("SET FOREIGN_KEY_CHECKS=1;"))

    # 세션 커밋
    session.commit()
except SQLAlchemyError as e:
    print(f"An error occurred: {e}")
    session.rollback()  # 오류 발생시 롤백
finally:
    session.close()  # 세션 닫기

print("Database update complete.")

1
2
Database update complete.


In [43]:
# SQL 쿼리 실행
query = "SELECT * FROM news_data"
news = pd.read_sql(query, engine)

# 데이터 출력
news.head()

,id,comp_id,title,link,date,sentiment,confidence
0,1,1,"코스피, ASML 쇼크에 2610선 '미끌'…'5만전자' 마감",http://www.newsprime.co.kr/news/article.html?n...,2024-10-16 16:56:00,negative,0.430158
1,2,1,디아이‧롯데칠성‧신라젠 등,https://www.etoday.co.kr/news/view/2410222,2024-10-16 16:56:00,neutral,0.996713
2,3,1,삼성전자 위기론 확산... 이재용 회장 입에 쏠린 눈,http://www.opinionnews.co.kr/news/articleView....,2024-10-16 16:56:00,negative,0.860533
3,4,1,미 반도체 급락에 SK하이닉스·삼성전자↓… 코스피 2610선 마감,https://n.news.naver.com/mnews/article/417/000...,2024-10-16 16:55:00,negative,0.952762
4,5,1,ASML쇼크 버틴 애플…힘 못받는 韓 관련주들,https://n.news.naver.com/mnews/article/215/000...,2024-10-16 16:54:00,negative,0.947249


In [44]:
# SQL 쿼리 실행
query = "SELECT * FROM company_list"
comp = pd.read_sql(query, engine)

# 데이터 출력
comp.head()

,id,name
0,1,삼성전자
1,2,SK하이닉스
2,3,LG에너지솔루션
3,4,삼성바이오로직스
4,5,현대차


In [45]:
comp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      100 non-null    int64 
 1   name    100 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [ ]:
from sqlalchemy import create_engine

# 데이터베이스 연결 설정
database_username = 'root'
database_password = 'chusong3112!'
database_ip       = '127.0.0.1'
database_name     = 'sentimentdb'
database_connection = f'mysql+mysqlconnector://{database_username}:{database_password}@{database_ip}/{database_name}'

# 데이터베이스 엔진 생성
engine = create_engine(database_connection)

# company_list DataFrame을 MySQL 데이터베이스에 저장
company_list_df.to_sql('company_list', con=engine, if_exists='append', index=False)

# news_data DataFrame을 MySQL 데이터베이스에 저장
news_df.to_sql('news_data', con=engine, if_exists='append', index=False)

### mySQL DB 테이블 생성 쿼리

In [ ]:
create TABLE company_list (
    id INT PRIMARY KEY,
    name VARCHAR(50) NOT NULL
) COMMENT '기업 테이블';

CREATE TABLE news_data (
    id INT PRIMARY KEY AUTO_INCREMENT,
    comp_id INT NOT NULL,
    title VARCHAR(512) NOT NULL,
    link VARCHAR(512) NOT NULL,
    date DATETIME,
    sentiment ENUM('positive', 'negative', 'neutral') NOT NULL,
    confidence FLOAT NOT NULL,
    FOREIGN KEY (comp_id) REFERENCES company_list(id)
) COMMENT '데이터 테이블';